In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sympy import symbols, Function, Derivative, Eq, dsolve, sin, cos, sqrt, simplify, init_printing
import sympy as sp
import os

def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')
clear_screen()

# Initialize pretty printing for symbolic mathematics
init_printing()

def derive_euler_lagrange():
    """Symbolically derive the Euler-Lagrange equation for the harmonic oscillator"""
    print("Deriving the Euler-Lagrange equation for the harmonic oscillator:\n")
    
    # Define symbols
    t = symbols('t')
    m, k = symbols('m k', positive=True)
    x = Function('x')(t)
    x_dot = Derivative(x, t)
    x_ddot = Derivative(x, t, t)
    
    # Define Lagrangian
    T = m * x_dot**2 / 2  # Kinetic energy
    V = k * x**2 / 2      # Potential energy
    L = T - V             # Lagrangian
    
    print("Lagrangian L = T - V =")
    sp.pprint(L)
    print()
    
    # Euler-Lagrange equation
    dL_dx = sp.diff(L, x)
    dL_dxdot = sp.diff(L, x_dot)
    el_eq = Eq(dL_dxdot.diff(t) - dL_dx, 0)
    
    print("Euler-Lagrange equation:")
    sp.pprint(el_eq)
    print()
    
    # Simplify the equation
    el_eq_simplified = simplify(el_eq)
    print("Simplified Euler-Lagrange equation:")
    sp.pprint(el_eq_simplified)
    print()
    
    return el_eq_simplified

def solve_harmonic_oscillator():
    """Solve the harmonic oscillator equation symbolically"""
    print("Solving the harmonic oscillator equation:\n")
    
    t = symbols('t')
    m, k = symbols('m k', positive=True)
    x = Function('x')(t)
    omega = symbols('ω', positive=True)
    
    # Harmonic oscillator equation: m*x''(t) + k*x(t) = 0
    # Which can be written as: x''(t) + ω²*x(t) = 0, where ω = sqrt(k/m)
    ode = Eq(Derivative(x, t, t) + (k/m)*x, 0)
    
    print("Harmonic oscillator equation:")
    sp.pprint(ode)
    print()
    
    # Solve the ODE
    solution = dsolve(ode, x)
    print("General solution:")
    sp.pprint(solution)
    print()
    
    return solution

def calculate_action(path, t_points, m, k):
    """Calculate the action for a given path"""
    # Compute derivatives using finite differences
    dx_dt = np.gradient(path, t_points)
    
    # Kinetic energy: T = (1/2)*m*(dx/dt)^2
    T = 0.5 * m * dx_dt**2
    
    # Potential energy: V = (1/2)*k*x^2
    V = 0.5 * k * path**2
    
    # Lagrangian: L = T - V
    L = T - V
    
    # Action: S = ∫L dt
    action = np.trapezoid(L, t_points)
    
    return action

def harmonic_solution(t, A, B, omega):
    """Analytical solution for harmonic oscillator"""
    return A * np.cos(omega * t) + B * np.sin(omega * t)

def main():
    # Parameters
    m = 1.0  # mass
    k = 2.0  # spring constant
    omega = np.sqrt(k / m)  # natural frequency
    
    # Time interval: choose t_end less than π/ω to ensure action is bounded below
    t_start = 0.0
    t_end = 2.0  # π/ω ≈ 2.22, so 2.0 < 2.22
    n_points = 100
    t_points = np.linspace(t_start, t_end, n_points)
    
    # Boundary conditions: set x_end consistent with equation of motion for consistency
    x_start = 1.0
    # Use analytical solution with A=1, B=0 for initial condition
    x_end = np.cos(omega * t_end)  # so x_end = cos(ω*t_end)
    
    print(f"Parameters: m={m}, k={k}, ω={omega:.3f}")
    print(f"Time interval: t_start={t_start}, t_end={t_end}")
    print(f"Boundary conditions: x_start={x_start}, x_end={x_end:.3f}")
    print()

    # 1. Symbolic derivation
    print("="*60)
    print("SYMBOLIC DERIVATION")
    print("="*60)
    derive_euler_lagrange()
    solution = solve_harmonic_oscillator()
    
    # 2. Find the optimal path using optimization
    print("="*60)
    print("NUMERICAL OPTIMIZATION")
    print("="*60)
    
    # Initial guess (straight line between endpoints)
    initial_guess = np.linspace(x_start, x_end, n_points)
    
    # Define the objective function (action to minimize)
    def objective(path_values):
        return calculate_action(path_values, t_points, m, k)
    
    # Constraint: fixed endpoints
    constraints = [
        {'type': 'eq', 'fun': lambda x: x[0] - x_start},
        {'type': 'eq', 'fun': lambda x: x[-1] - x_end}
    ]
    
    # Perform optimization
    result = minimize(objective, initial_guess, constraints=constraints, method='SLSQP', options={'maxiter': 1000})
    
    if result.success:
        optimal_path = result.x
        optimal_action = result.fun
        print(f"Optimization successful! Minimum action: {optimal_action:.6f}")
    else:
        print("Optimization failed:", result.message)
        return
    
    # 3. Compare with analytical solution
    print("\n" + "="*60)
    print("COMPARISON WITH ANALYTICAL SOLUTION")
    print("="*60)
    
    # Solve for A and B using boundary conditions
    cos_start = np.cos(omega * t_start)
    sin_start = np.sin(omega * t_start)
    cos_end = np.cos(omega * t_end)
    sin_end = np.sin(omega * t_end)
    
    # Matrix equation: M * [A, B]^T = [x_start, x_end]^T
    M = np.array([[cos_start, sin_start], [cos_end, sin_end]])
    det = cos_start * sin_end - sin_start * cos_end  # determinant: sin(ω*(t_end-t_start))
    
    if np.abs(det) < 1e-10:
        # Check if endpoints are consistent
        if np.abs(cos_start * x_end - cos_end * x_start) < 1e-10:
            # Consistent case: choose B=0 arbitrarily
            if np.abs(cos_start) > 1e-10:
                A = x_start / cos_start
            else:
                A = x_end / cos_end
            B = 0.0
        else:
            # Inconsistent endpoints: use least squares or default
            A = x_start
            B = 0.0
    else:
        # Solve for A and B
        A = (x_start * sin_end - x_end * sin_start) / det
        B = (x_end * cos_start - x_start * cos_end) / det
    
    analytical_path = harmonic_solution(t_points, A, B, omega)
    analytical_action = calculate_action(analytical_path, t_points, m, k)
    
    print(f"Analytical coefficients: A={A:.6f}, B={B:.6f}")
    print(f"Analytical action: {analytical_action:.6f}")
    print(f"Numerical action:  {optimal_action:.6f}")
    print(f"Difference:        {abs(analytical_action - optimal_action):.6f}")
    
    # 4. Visualization
    plt.figure(figsize=(12, 8))
    
    # Plot the paths
    plt.subplot(2, 1, 1)
    plt.plot(t_points, optimal_path, 'b-', linewidth=2, label='Numerical optimal path')
    plt.plot(t_points, analytical_path, 'r--', linewidth=2, label='Analytical solution')
    plt.plot([t_start, t_end], [x_start, x_end], 'ko', markersize=8, label='Boundary conditions')
    plt.xlabel('Time')
    plt.ylabel('Position')
    plt.title('Optimal Path for Harmonic Oscillator')
    plt.legend()
    plt.grid(True)
    
    # Plot the Lagrangian along the path
    plt.subplot(2, 1, 2)
    dx_dt_num = np.gradient(optimal_path, t_points)
    T_num = 0.5 * m * dx_dt_num**2
    V_num = 0.5 * k * optimal_path**2
    L_num = T_num - V_num
    
    dx_dt_ana = np.gradient(analytical_path, t_points)
    T_ana = 0.5 * m * dx_dt_ana**2
    V_ana = 0.5 * k * analytical_path**2
    L_ana = T_ana - V_ana
    
    plt.plot(t_points, L_num, 'b-', linewidth=2, label='Numerical Lagrangian')
    plt.plot(t_points, L_ana, 'r--', linewidth=2, label='Analytical Lagrangian')
    plt.xlabel('Time')
    plt.ylabel('Lagrangian (T - V)')
    plt.title('Lagrangian Along the Path')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # 5. Show that other paths have higher action
    print("\n" + "="*60)
    print("DEMONSTRATING THAT OTHER PATHS HAVE HIGHER ACTION")
    print("="*60)
    
    # Test some alternative paths
    test_paths = []
    test_actions = []
    
    # Straight line path
    straight_path = np.linspace(x_start, x_end, n_points)
    straight_action = calculate_action(straight_path, t_points, m, k)
    test_paths.append(straight_path)
    test_actions.append(straight_action)
    print(f"Straight line action: {straight_action:.6f}")
    
    # Parabolic path
    parabolic_path = x_start + (x_end - x_start) * ((t_points - t_start) / (t_end - t_start))**2
    parabolic_action = calculate_action(parabolic_path, t_points, m, k)
    test_paths.append(parabolic_path)
    test_actions.append(parabolic_action)
    print(f"Parabolic path action: {parabolic_action:.6f}")
    
    # Sinusoidal path with wrong frequency
    wrong_omega = omega * 0.7
    # Solve for coefficients with wrong frequency
    cos_start_w = np.cos(wrong_omega * t_start)
    sin_start_w = np.sin(wrong_omega * t_start)
    cos_end_w = np.cos(wrong_omega * t_end)
    sin_end_w = np.sin(wrong_omega * t_end)
    det_w = cos_start_w * sin_end_w - sin_start_w * cos_end_w
    if np.abs(det_w) < 1e-10:
        A_w = x_start
        B_w = 0.0
    else:
        A_w = (x_start * sin_end_w - x_end * sin_start_w) / det_w
        B_w = (x_end * cos_start_w - x_start * cos_end_w) / det_w
    wrong_path = harmonic_solution(t_points, A_w, B_w, wrong_omega)
    wrong_action = calculate_action(wrong_path, t_points, m, k)
    test_paths.append(wrong_path)
    test_actions.append(wrong_action)
    print(f"Wrong frequency action: {wrong_action:.6f}")
    
    print(f"Optimal path action: {optimal_action:.6f}")
    
    # Plot comparison of paths
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 1, 1)
    plt.plot(t_points, optimal_path, 'k-', linewidth=3, label='Optimal path')
    plt.plot(t_points, straight_path, 'b--', linewidth=2, label='Straight line')
    plt.plot(t_points, parabolic_path, 'g--', linewidth=2, label='Parabolic')
    plt.plot(t_points, wrong_path, 'r--', linewidth=2, label='Wrong frequency')
    plt.plot([t_start, t_end], [x_start, x_end], 'ko', markersize=8, label='Boundary conditions')
    plt.xlabel('Time')
    plt.ylabel('Position')
    plt.title('Comparison of Different Paths')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(2, 1, 2)
    labels = ['Optimal', 'Straight', 'Parabolic', 'Wrong frequency']
    actions = [optimal_action, straight_action, parabolic_action, wrong_action]
    plt.bar(labels, actions)
    plt.ylabel('Action')
    plt.title('Action Values for Different Paths')
    plt.grid(True, axis='y')
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()